In [2]:
import kitorch as kt
from kitorch import nn,optim,functional as F

ModuleNotFoundError: No module named 'kitorch'

In [ ]:
import numpy as np
import torch
# from torch import nn,optim
# from torch.nn import functional as F
from torchvision import datasets, transforms

In [ ]:
BATCH_SIZE=256
EPOCHS=5 # 总共训练批次

In [ ]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self): 
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(1,10,5) # 10, 24x24
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10,20,3) # 128, 10x10
        self.bn2 = nn.BatchNorm2d(20)
        self.fc1 = nn.Linear(20*10*10,200)
        self.bn3 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200,10)
        
    def forward(self,x:mt.Tensor):
        # x： 512-1-28-28
        batch_size = x.shape[0]
        out = self.conv1(x)     # 512-10-24-24
        out = self.bn1(out)
        out = F.relu(out)
        out = F.maxpool2d(out,(2,2)) # 512-10-12-12
        out = self.conv2(out) # 512-20-10-10
        out = self.bn2(out)
        out = F.relu(out)
        out = out.reshape((batch_size,-1))
        out = self.fc1(out)
        out = self.bn3(out)
        out = F.relu(out)
        out = self.fc2(out)
        return F.log_softmax(out,dim=1)

In [ ]:

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data,target = mt.from_numpy(data.numpy().astype(np.float64)),target.numpy()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%100 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model,test_loader):
    test_loss = 0
    correct = 0
    model.eval()
    for data, target in test_loader:
        data,target = mt.from_numpy(data.numpy().astype(np.float64)),target.numpy()
        output = model(data)
        test_loss += F.nll_loss(output, target).item() # 将一批的损失相加
        result = output.data.argmax(axis=1)
        correct += (result == target).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.6f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
%%time
EPOCHS = 20
model = ConvNet()
#测试１：
# lr = [(3,0.05),(5,0.02),(7,0.001),(10,0.005),(15,0.001),(20,0.0005)]
# optimizer = optim.SGD(model.parameters(),lr,momentum=0.9)

# 测试2
# lr = [(3,0.05),(5,0.02),(7,0.001),(10,0.005),(15,0.001),(20,0.0005)]
# optimizer = optim.Adagrad(model.parameters(),lr=lr) 

# 128, 5 epoch 可达到99%
optimizer = optim.Adadelta(model.parameters())

# 测试失败
# optimizer = optim.RMSprop(model.parameters(),beta=0.5)

# 256, 20次可达到99.11%
# optimizer = optim.Adam(model.parameters())

for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test(model,test_loader)